In [1]:
import pandas as pd
csv_data1 = pd.read_csv('D:\\College\\Sem2\\Dissertation\\Hotel_Reviews.csv')

In [2]:
def filter_country(address):
    country = address.split()[-1]
    if (country == 'Kingdom'):
                country = 'United Kingdom'
    return country

csv_data1["Country_Name"] = csv_data1["Hotel_Address"].apply(lambda x: filter_country(x))

In [3]:
csv_data1['Hotel_Name'].value_counts()

Britannia International Hotel Canary Wharf           4789
Strand Palace Hotel                                  4256
Park Plaza Westminster Bridge London                 4169
Copthorne Tara Hotel London Kensington               3578
DoubleTree by Hilton Hotel London Tower of London    3212
                                                     ... 
Ibis Styles Milano Palmanova                           12
Le Lavoisier                                           12
Mercure Paris Porte d Orleans                          10
Hotel Wagner                                           10
Hotel Gallitzinberg                                     8
Name: Hotel_Name, Length: 1492, dtype: int64

In [4]:
#p = csv_data1.head(1000)
p = csv_data1
p['Hotel_Name'].value_counts()

Britannia International Hotel Canary Wharf           4789
Strand Palace Hotel                                  4256
Park Plaza Westminster Bridge London                 4169
Copthorne Tara Hotel London Kensington               3578
DoubleTree by Hilton Hotel London Tower of London    3212
                                                     ... 
Ibis Styles Milano Palmanova                           12
Le Lavoisier                                           12
Mercure Paris Porte d Orleans                          10
Hotel Wagner                                           10
Hotel Gallitzinberg                                     8
Name: Hotel_Name, Length: 1492, dtype: int64

In [5]:
csv_data1.head(1)

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Country_Name
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,Netherlands


In [6]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

In [7]:
def summarize_based_on_hotels(hotel_names,csv_data1,x):
    p = csv_data1[csv_data1['Hotel_Name'].isin(hotel_names)]
    p['Hotel_Name'].value_counts()
    t = p.groupby(['Hotel_Name','Country_Name'])[x].apply('.'.join).reset_index()
    return t

In [9]:

def find_similarity(sentence1, sentence2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sentence1 = [a.lower() for a in sentence1]
    sentence2 = [a.lower() for a in sentence2]
 
    all_words = list(set(sentence1 + sentence2))
 
    vector_first = [0] * len(all_words)
    vector_second = [0] * len(all_words)
 
    for a in sentence1:
        if a in stopwords:
            continue
        vector_first[all_words.index(a)] += 1
 
    for a in sentence2:
        if a in stopwords:
            continue
        vector_second[all_words.index(a)] += 1
 
    return 1 - cosine_distance(vector_first, vector_second)
 
def create_similarity(texts, stop_words):
    # Build a blank matrix
    sim_matrix = np.zeros((len(texts), len(texts)))
 
    for idx1 in range(len(texts)):
        for idx2 in range(len(texts)):
            if idx1 == idx2: #neglect when sentences are similar
                continue 
            sim_matrix[idx1][idx2] = find_similarity(texts[idx1], texts[idx2], stop_words)

    return sim_matrix


def generate_summary(text):
    stop_words = stopwords.words('english')
    summary = []
    top_n=2
    fdata = text
    article = fdata.split(". ")
    texts = []

    for sentence in article:
        texts.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    texts.pop()
    

    #Build the matrix
    text_similarity_martix = create_similarity(texts, stop_words)
    print(text_similarity_martix)
   #Rank the sentences
    text_similarity_graph = nx.from_numpy_array(text_similarity_martix)
    scores = nx.pagerank(text_similarity_graph)

    # Sort the sentences
    order_sentence = sorted(((scores[i],s) for i,s in enumerate(texts)), reverse=True)    
    print("Indexes of top order sentences are ", order_sentence)    

    for i in range(top_n):
        summary.append(" ".join(order_sentence[i][1]))

    return summary

#Provide list to hotels to be summarize
hotel_name_list = ['Hotel Arena','1K Hotel']
t = summarize_based_on_hotels(hotel_name_list,csv_data1,'Negative_Review')
p = summarize_based_on_hotels(hotel_name_list,csv_data1,'Positive_Review')
#t['Summary_Negative_Review'] = t.apply(lambda row : generate_summary(row['Negative_Review']), axis = 1) 
p['Summary_Positive_Review'] = p.apply(lambda row : generate_summary(row['Positive_Review']), axis = 1) 

[[0.         0.11785113 0.24019223 ... 0.         0.16666667 0.        ]
 [0.11785113 0.         0.22645541 ... 0.         0.         0.        ]
 [0.24019223 0.22645541 0.         ... 0.         0.16012815 0.        ]
 ...
 [0.         0.         0.         ... 0.         0.25819889 0.        ]
 [0.16666667 0.         0.16012815 ... 0.25819889 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
Indexes of top order sentences are  [(0.014032783375690307, ['Location', 'was', 'great', 'room', 'was', 'spacious', '']), (0.013492851429516109, ['Good', 'location', 'and', 'clean', 'rooms', '']), (0.013086751136824521, ['A', 'very', 'nice', 'hotel', 'at', 'a', 'central', 'location', 'Very', 'helpful', 'staff', 'and', 'a', 'nice', 'modern', 'clean', 'room', '']), (0.01276350971630862, ['location', 'room', 'size', '']), (0.012720055361245642, ['The', 'location', 'of', 'the', 'hotel', '']), (0.012140046873508907, ['Good', 'location', 'and', 'next', 'to

[[0.         0.11858541 0.30618622 ... 0.5        0.28867513 0.35355339]
 [0.11858541 0.         0.19364917 ... 0.07905694 0.13693064 0.1677051 ]
 [0.30618622 0.19364917 0.         ... 0.20412415 0.23570226 0.28867513]
 ...
 [0.5        0.07905694 0.20412415 ... 0.         0.28867513 0.35355339]
 [0.28867513 0.13693064 0.23570226 ... 0.28867513 0.         0.40824829]
 [0.35355339 0.1677051  0.28867513 ... 0.35355339 0.40824829 0.        ]]
Indexes of top order sentences are  [(0.004868952468467998, ['The', 'staff', 'were', 'really', 'helpful', 'and', 'the', 'hotel', 'was', 'in', 'a', 'great', 'location', '']), (0.004706386875432794, ['The', 'room', 'was', 'lovely', 'great', 'hotel', 'beautiful', 'location', '']), (0.0044908404754543125, ['Great', 'location', 'lovely', 'helpful', 'staff', 'very', 'pretty', 'and', 'modern', 'hotel', '']), (0.004458777433785301, ['The', 'room', 'the', 'building', 'the', 'little', 'details', 'in', 'and', 'around', 'the', 'hotel', 'were', 'very', 'nice', 'S

In [10]:
p

,Hotel_Name,Country_Name,Positive_Review,Summary_Positive_Review
0,1K Hotel,France,Location good close to le Marais and 3e arron...,"[Location was great room was spacious , Good l..."
1,Hotel Arena,Netherlands,Only the park outside of the hotel was beauti...,[The staff were really helpful and the hotel w...


In [11]:
t['Positive_Review'] = p['Positive_Review']
t['Summary_Positive_Review'] = p['Summary_Positive_Review']

In [12]:
t[['Hotel_Name','Country_Name','Positive_Review','Summary_Positive_Review']]

,Hotel_Name,Country_Name,Positive_Review,Summary_Positive_Review
0,1K Hotel,France,Location good close to le Marais and 3e arron...,"[Location was great room was spacious , Good l..."
1,Hotel Arena,Netherlands,Only the park outside of the hotel was beauti...,[The staff were really helpful and the hotel w...


In [13]:
t['Summary_Positive_Review'][1]

['The staff were really helpful and the hotel was in a great location ',
 'The room was lovely great hotel beautiful location ']

In [14]:
print(" ")
t['Positive_Review'][1]

' Only the park outside of the hotel was beautiful . No real complaints the hotel was great great location surroundings rooms amenities and service Two recommendations however firstly the staff upon check in are very confusing regarding deposit payments and the staff offer you upon checkout to refund your original payment and you can make a new one Bit confusing Secondly the on site restaurant is a bit lacking very well thought out and excellent quality food for anyone of a vegetarian or vegan background but even a wrap or toasted sandwich option would be great Aside from those minor minor things fantastic spot and will be back when i return to Amsterdam . Location was good and staff were ok It is cute hotel the breakfast range is nice Will go back . Great location in nice surroundings the bar and restaurant are nice and have a lovely outdoor area The building also has quite some character . Amazing location and building Romantic setting . Good restaurant with modern design great chill

In [15]:
t['Positive_Review'][1]

' Only the park outside of the hotel was beautiful . No real complaints the hotel was great great location surroundings rooms amenities and service Two recommendations however firstly the staff upon check in are very confusing regarding deposit payments and the staff offer you upon checkout to refund your original payment and you can make a new one Bit confusing Secondly the on site restaurant is a bit lacking very well thought out and excellent quality food for anyone of a vegetarian or vegan background but even a wrap or toasted sandwich option would be great Aside from those minor minor things fantastic spot and will be back when i return to Amsterdam . Location was good and staff were ok It is cute hotel the breakfast range is nice Will go back . Great location in nice surroundings the bar and restaurant are nice and have a lovely outdoor area The building also has quite some character . Amazing location and building Romantic setting . Good restaurant with modern design great chill